# Translate any video into any language

## Preparation 

In [ ]:
# imports 
# this might be necessary if you are using docker the way I am
import os 
os.chdir('video-translation')
from utils import *

In [ ]:
# set url for video
url = "https://www.youtube.com/watch?v=gbNzxvz2_H4"

## Process Audio

In [ ]:
# download audio
audio_file = download_audio(url)

In [ ]:
# transcribe audio
transcription = transcribe_audio(audio_file)
with open('transcription.txt', 'w') as f:
    f.write(transcription['text'])
start, end, english = extract_transcription(transcription)

## Translate Trasncript

In [ ]:
# summarize transcription
# you might want to generate a long context Ollama model first
summarized_transcription = summarize_transcription(transcription, "8k-qwen2.5:7b")

In [ ]:
# translate to any langugaes
video_context = "A Minecraft Movie | Official Trailer (我的世界真人电影|预告)" # you can use this to make the model more accurate
system_prompt = f"""
Translate the following English text to Simplified Chinese (Mandarin). The text is part of a {video_context} video script. 
Here is a short summary of the video content: {summarized_transcription}
Ensure the translation feels natural and culturally localized, avoiding direct English phrasing where possible. 
Maintain a tone that is light, friendly, and suitable for the context, but not overly cheerful. Keep the translation concise. 
Only reply with the translation text, nothing else. 
Here is the English text to translate:
""" 

chinese = translate_transcription(english, system_prompt, "qwen2.5:7b")

## Save to SRT

In [ ]:
# save srt files
filename = "".join(audio_file.split('.')[:-1])

save_srt(f'{filename}.en.srt', start, end, english)
save_srt(f'{filename}.zh.srt', start, end, chinese)

In [ ]:
# download video
os.remove(audio_file)
video_file = download_video(url)

In [ ]:
# after manual correction, update the srt files
# start, end, english = load_srt(f'{filename}.en.srt')
start, end, chinese = load_srt(f'{filename}.zh.srt')

save_srt(f'{filename}.davinci.zh.srt', start, end, chinese, True)
# save_srt(f'{filename}.davinci.en.srt', start, end, english, True)
# save_bilingual_srt(f'{filename}.zh.en.srt', start, end, english, chinese)
# save_bilingual_srt(f'{filename}.davinci.zh.en.srt', start, end, chinese, english, True)